In [ ]:
import numpy as np
import os
import py7zr

In [ ]:
img_size = 512
path = os.getcwd() + "/data/" + str(img_size)
train_path = path + "/train"
test_path  = path + "/test"

from imgProc import preprocess_img

In [ ]:
def calc_NCD(KX, y_name, y_label):
    KY = os.path.getsize(train_path + "/archives/" + y_label + "/" + y_name + ".7z")

    x_text = ""
    with open(test_path + "/text files/TEST/test.txt", "rb") as f:
        x_text = f.read()

    y_text = ""
    with open(train_path + "/text files/" + y_label + "/" + y_name + ".txt", "rb") as f:
        y_text = f.read()

    xy_text = x_text + y_text
    text_file = open(test_path + "/text files/TEST/XY.txt", "wb")
    text_file.write(xy_text)
    text_file.close()

    with py7zr.SevenZipFile(test_path + "/archives/TEST/XY.7z", 'w') as archive:
        archive.write( "XY.txt")
    
    K_XY = os.path.getsize(test_path + "/archives/TEST/XY.7z")
    
    if KX < KY:
        min_K = KX
        max_K = KY
    else:
        min_K = KY
        max_K = KX
        
    return (K_XY - min_K) / max_K

In [ ]:
def predict(directory, img_path, img_size, normal_size, pneumonia_size):
    preprocess_img(directory, img_path, img_size, "TEST", "test")
    KX = os.path.getsize(test_path + "/archives/TEST/test.7z")
    
    P = []
    N = []
    os.chdir(test_path + "/text files/TEST")

    for i in range(normal_size):           
        N.append(calc_NCD(KX, str(i+1), "NORMAL"))

    for i in range(pneumonia_size):
        P.append(calc_NCD(KX, str(i+1), "PNEUMONIA"))
        
    os.chdir(test_path[:test_path.rfind('/data')])
    
    if np.mean(N) < np.mean(P):
        return 0
    else:
        return 1

In [ ]:
normal_size = 100
pneumonia_size = 100

In [ ]:
label = "NORMAL"
count = 0
summ = 0 # сумма всех положительных вердиктов алгоритма, при классификации снимков без пневмонии 
         # равна числу ложноположительных вердиктов (FP)
for img in os.listdir(test_path + "/raw images/" + label):
    result = predict(
        test_path, 
        test_path + "/raw images/" + label + "/" + img, 
        img_size, 
        normal_size, 
        pneumonia_size
        )
    summ += result
    print(result)
    count += 1

In [ ]:
print(count, summ)

In [ ]:
label = "PNEUMONIA"
count = 0
summ = 0 # сумма всех положительных вердиктов алгоритма, при классификации снимков с пневмонией
         # равна числу истинно положительных вердиктов (TP)

for img in os.listdir(test_path + "/raw images/" + label):
    result = predict(
        test_path, 
        test_path + "/raw images/" + label + "/" + img, 
        img_size, 
        normal_size, 
        pneumonia_size
        )
    summ += result
    print(result)
    count += 1

In [ ]:
print(count, summ)